In [1]:
import pandas as pd
from pathlib import Path
import pydub
from tqdm.auto import tqdm

In [2]:
abo16_list = pd.read_csv("../data/abo16_list.txt")
anmp3_dir = Path("../data/anmp3")
anmp3_dir.mkdir(parents=True, exist_ok=True)

In [3]:
abo16_list.head()

,filename,lang
0,c5008-2201151100.mp4,Seediq
1,c5008-2201150800.mp4,Thau
2,c5008-2201142000.mp4,Saysiyat
3,c5008-2201141100.mp4,Rukai
4,c5008-2201132000.mp4,Bunun


In [6]:
def make_mp3_path(video_name):
    mp3_dir = Path("../data/mp3")    
    video_path = mp3_dir/ video_name.replace(".mp4", ".mp3")
    return video_path

In [7]:
mp3_dir = Path("../data/mp3")

In [8]:
import numpy as np

In [9]:
misc_index = np.arange(abo16_list.shape[0])
np.random.seed(123)
np.random.shuffle(misc_index,)
# misc_index = misc_index[:9]
misc_index = []

In [ ]:
# thau_list = abo16_list.loc[abo16_list.lang=='Thau']
paiwan_list = abo16_list.loc[abo16_list.lang=='Paiwan']

In [15]:
for idx in tqdm(range(abo16_list.shape[0])):
# for idx in tqdm(range(paiwan_list.shape[0])):
    basename = abo16_list.iloc[idx].filename
    out_path = anmp3_dir / basename.replace(".mp4", ".mp3")    
        
    lang = abo16_list.iloc[idx].lang
    mp3_path = make_mp3_path(basename)
    mp3 = pydub.AudioSegment.from_mp3(mp3_path)
    anchor_path = "../data/sandbox/anchor_segs/" + basename.replace(".mp4", ".anchor.csv")
    anchor_seg = pd.read_csv(anchor_path, index_col=0)
    an_offsets = anchor_seg.loc[anchor_seg.is_anchor].offset.tolist()
    misc_offsets = anchor_seg.iloc[20:].loc[~anchor_seg.is_anchor].offset.tolist()
    if len(an_offsets) < 1:
        print("INFO: fewer than 10 anchor footage detected in ", basename)
    misc_offsets = misc_offsets[:len(an_offsets)]
    concat = pydub.AudioSegment.empty()
    
    if idx in misc_index:
        misc = pydub.AudioSegment.empty()
    else:
        misc = None
        
    cursor = 0
    for offset_i, offset_x in enumerate(an_offsets):    
        if offset_x < cursor: continue    
        if misc is not None:            
            misc_x = misc_offsets[offset_i]            
            misc += mp3[misc_x*1000: (misc_x+5)*1000]
        concat += mp3[offset_x*1000: (offset_x+5)*1000]
        cursor = offset_x+5
    
    out_path = anmp3_dir / basename.replace(".mp4", ".mp3")
    concat.export(out_path)
    if misc is not None:
        misc_path = anmp3_dir / ("misc_" + basename.replace(".mp4", ".mp3"))
        misc.export(misc_path)
    

  0%|          | 0/9 [00:00<?, ?it/s]

INFO: fewer than 10 anchor footage detected in  c5008-2201082000.mp4
INFO: fewer than 10 anchor footage detected in  c5008-2112252000.mp4
INFO: fewer than 10 anchor footage detected in  c5008-2111272000.mp4


## Anchor stat

In [27]:
anchor_map = {}
for idx in tqdm(range(abo16_list.shape[0])):
# for idx in tqdm(range(paiwan_list.shape[0])):
    basename = abo16_list.iloc[idx].filename
    out_path = anmp3_dir / basename.replace(".mp4", ".mp3")    
        
    lang = abo16_list.iloc[idx].lang        
    anchor_path = "../data/sandbox/anchor_segs/" + basename.replace(".mp4", ".anchor.csv")
    anchor_seg = pd.read_csv(anchor_path, index_col=0)
    anchor_name = anchor_seg.loc[anchor_seg.is_anchor].anchor_name
    try:
        counter = anchor_map.setdefault(lang, Counter())
        counter[anchor_name[0]] += 1
    except Exception:
        pass

  0%|          | 0/144 [00:00<?, ?it/s]

In [30]:
{k: len(v) for k, v in anchor_map.items()}

{'Seediq': 2,
 'Thau': 1,
 'Saysiyat': 4,
 'Rukai': 3,
 'Bunun': 1,
 'Truku': 1,
 'Sakizaya': 2,
 'Cou': 3,
 'Yami': 1,
 'Kavalan': 1,
 'Amis': 1,
 'Atayal': 2,
 'Pinuyumayan': 1,
 'Kanakanavu': 1,
 "Hla'alua": 1,
 'Paiwan': 1}